In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np
from operator import itemgetter
import time
from kuka import Kuka
from openshowvar import *

print('go')
robot = openshowvar(ip = '192.168.1.2', port = 7000)
kuka = Kuka(robot)

kuka.read_cartessian()

kuka.set_base(17)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(100)
print('ready')


trajectory_arr = []
trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
trajectory_arr.append(np.array([0, 0, -50, 0, 0, 180]))
trajectory_arr = np.array(trajectory_arr)
# Отправка траектории роботу
kuka.lin_continuous(trajectory_arr) 
kuka.open_grip()
            
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
model = YOLO("Battery_1.pt")

bounding_box_annotator = sv.BoundingBoxAnnotator()
#label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER)
percentage_bar_annotator = sv.BoundingBoxAnnotator()

def speed(Hz):
    second=(35/(Hz/2.8))/2
    return second


while True:
    ret, frame = cap.read()
    frame=frame[120:370,:]
    result = model(frame, agnostic_nms=True)[0]
    detections = sv.Detections.from_ultralytics(result)
    #print(detections) 
    coord_center=[]
    for i in range (len(detections)):
        x=(detections.xyxy[i][0]+detections.xyxy[i][2])/2
        y=(detections.xyxy[i][1]+detections.xyxy[i][3])/2
        y_mm=210/250*(250-y)
        coord_center.append([x,y,y_mm])
    #print(coord_center)
        
    #frame 250
    #conveer 204
    coord_center=np.array(coord_center)
    if coord_center.size != 0:
        x_sort=sorted(coord_center,key=itemgetter(0))
        #print(x_sort)
        currentClass = detections.class_id[0]
        if x_sort[0][0]<=200:
            second_detection=int(time.time())
            second=speed(7.4)
            print(second)
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([0, x_sort[0][2], -120, 0, 0, 180]))
            trajectory_arr = np.array(trajectory_arr)
            # Отправка траектории роботу
            kuka.lin_continuous(trajectory_arr)
            time.sleep(second)
            kuka.close_grip()
            if currentClass==1:
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([0, x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, -100, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
                kuka.open_grip()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([-330, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, 0, -50, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 

    
    
    for i in range (len(detections)):
        cv2.circle(frame, (int(coord_center[i][0]),int(coord_center[i][1])), 3, (118, 103, 154), 3)
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    #frame = label_annotator.annotate(
    #    scene=frame,
    #    detections=detections
    #)
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.line(frame, (200,0), (200, 480), (0, 255, 0), 5)
    
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

go
ready


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
Using TensorFlow backend.

0: 256x640 (no detections), 78.7ms
Speed: 3.0ms preprocess, 78.7ms inference, 18.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 25.1ms
Speed: 2.0ms preprocess, 25.1ms inference, 1.9ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 24.0ms
Speed: 3.0ms preprocess, 24.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 2.0ms 


0: 256x640 (no detections), 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 3.0ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 1.0ms preprocess, 23.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 2.6ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 2.0ms preprocess, 23.0ms i


0: 256x640 (no detections), 22.5ms
Speed: 1.0ms preprocess, 22.5ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 4.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.6ms
Speed: 2.0ms preprocess, 23.6ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 1.0ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.1ms
Speed: 2.0ms preprocess, 23.1ms inference, 1.6ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 21.5ms
Speed: 3.0ms preprocess, 21.5ms i


0: 256x640 (no detections), 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 1.0ms preprocess, 23.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.5ms
Speed: 2.0ms preprocess, 22.5ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 22.6ms
Speed: 2.0ms preprocess, 22.6ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 21.5ms
Speed: 2.9ms preprocess, 21.5ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 21.6ms
Speed: 3.0ms preprocess, 21.6ms i

...


0: 256x640 (no detections), 23.1ms
Speed: 4017.9ms preprocess, 23.1ms inference, 1.9ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 25.2ms
Speed: 2.0ms preprocess, 25.2ms inference, 2.9ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 3.1ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 24.1ms
Speed: 3.0ms preprocess, 24.1ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 4.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 24.6ms
Speed: 2.0ms preprocess, 24.6ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 25.1ms
Speed: 2.0ms preprocess, 25.1ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 24.0ms
Speed: 1.0ms preprocess, 24.0m


0: 256x640 1 battery_plus, 25.0ms
Speed: 2.0ms preprocess, 25.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 25.0ms
Speed: 2.0ms preprocess, 25.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 25.0ms
Speed: 2.0ms preprocess, 25.0ms inference, 4.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.6ms
Speed: 2.0ms preprocess, 24.6ms inference, 4.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.3ms
Speed: 2.0ms preprocess, 24.3ms inference, 2.9ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 25.0ms
Speed: 2.0ms preprocess, 25.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 2.9ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.5ms
Speed: 2.0ms preprocess, 24.5ms inference

Speed: 2.0ms preprocess, 26.1ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 26.0ms
Speed: 3.0ms preprocess, 26.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.1ms
Speed: 3.0ms preprocess, 24.1ms inference, 3.9ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.9ms
Speed: 1.0ms preprocess, 24.9ms inference, 3.8ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.0ms
Speed: 4.0ms preprocess, 24.0ms inference, 5.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 26.0ms
Speed: 2.0ms preprocess, 26.0ms inference, 5.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 25.2ms
Speed: 2.0ms preprocess, 25.2ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 23.6ms
Speed: 2.1ms preprocess, 23.6ms inference, 6.0ms postprocess per image at sh


0: 256x640 1 battery_plus, 25.0ms
Speed: 2.1ms preprocess, 25.0ms inference, 4.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.2ms
Speed: 1.0ms preprocess, 24.2ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 3.5ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 23.0ms
Speed: 2.0ms preprocess, 23.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.0ms
Speed: 1.0ms preprocess, 24.0ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.5ms
Speed: 1.0ms preprocess, 24.5ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 25.5ms
Speed: 2.0ms preprocess, 25.5ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 battery_plus, 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference

6.621621621621621


In [1]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np
from operator import itemgetter
import time
from kuka import Kuka
from openshowvar import *
'''
print('go')
robot = openshowvar(ip = '192.168.1.2', port = 7000)
kuka = Kuka(robot)

kuka.read_cartessian()

kuka.set_base(17)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(100)
print('ready')


trajectory_arr = []
trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
trajectory_arr.append(np.array([0, 0, -50, 0, 0, 180]))
trajectory_arr = np.array(trajectory_arr)
# Отправка траектории роботу
kuka.lin_continuous(trajectory_arr) 
kuka.open_grip()
'''            
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
model = YOLO("Battery_1.pt")

bounding_box_annotator = sv.BoundingBoxAnnotator()
#label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER)
percentage_bar_annotator = sv.BoundingBoxAnnotator()

def speed(Hz):
    second=(35/(Hz/2.8))/2
    return second


while True:
    ret, frame = cap.read()
    frame=frame[100:370,:]
    result = model(frame, agnostic_nms=True)[0]
    detections = sv.Detections.from_ultralytics(result)
    #print(detections) 
    coord_center=[]
    for i in range (len(detections)):
        x=(detections.xyxy[i][0]+detections.xyxy[i][2])/2
        y=(detections.xyxy[i][1]+detections.xyxy[i][3])/2
        y_mm=230/270*(270-y)
        coord_center.append([x,y,y_mm])
    print(coord_center)
        
    #frame 250
    #conveer 204
    coord_center=np.array(coord_center)
    if coord_center.size != 0:
        x_sort=sorted(coord_center,key=itemgetter(0))
        #print(x_sort)
        currentClass = detections.class_id[0]
        if x_sort[0][0]<=200:
            second_detection=int(time.time())
            second=speed(7.4)
            print(second)
            '''
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([0, x_sort[0][2], -120, 0, 0, 180]))
            trajectory_arr = np.array(trajectory_arr)
            # Отправка траектории роботу
            kuka.lin_continuous(trajectory_arr)
            time.sleep(second)
            kuka.close_grip()
            if currentClass==1:
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([0, x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, -100, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
                kuka.open_grip()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([-330, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, 0, -50, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
        '''
    
    
    for i in range (len(detections)):
        cv2.circle(frame, (int(coord_center[i][0]),int(coord_center[i][1])), 3, (118, 103, 154), 3)
        cv2.putText(frame, 'y_mm: {}'.format(coord_center[i][2]), (100, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    #frame = label_annotator.annotate(
    #    scene=frame,
    #    detections=detections
    #)
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.line(frame, (200,0), (200, 480), (0, 255, 0), 5)
    
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
Using TensorFlow backend.

0: 480x640 (no detections), 47.0ms
Speed: 3.0ms preprocess, 47.0ms inference, 77.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 20.0ms
Speed: 4.0ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



[]
[]


0: 480x640 1 battery_plus, 19.0ms
Speed: 3.0ms preprocess, 19.0ms inference, 68.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 20.0ms
Speed: 3.0ms preprocess, 20.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 19.0ms
Speed: 4.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 20.1ms


[[192.92079162597656, 249.66671752929688, 196.20983321578416]]
6.621621621621621
[[192.95034790039062, 249.55673217773438, 196.30352444118924]]
6.621621621621621
[[192.98980712890625, 249.67098999023438, 196.20619371202258]]
6.621621621621621


Speed: 3.0ms preprocess, 20.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 20.0ms
Speed: 1.9ms preprocess, 20.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 19.0ms


...

0: 480x640 1 battery_plus, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 battery_pluss, 20.0ms
Speed: 2.9ms preprocess, 20.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 20.0ms
Speed: 3.0ms preprocess, 20.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)



[[193.13458251953125, 249.20236206054688, 196.60539528175636]]
6.621621621621621
[[193.2450408935547, 249.1728515625, 196.63053385416666], [144.15234375, 222.06716918945312, 219.72055957935476]]
6.621621621621621
[[193.20281982421875, 249.26499938964844, 196.55203755696616]]
6.621621621621621


0: 480x640 2 battery_pluss, 19.0ms
Speed: 2.9ms preprocess, 19.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 battery_plus, 20.0ms
Speed: 2.0ms preprocess, 20.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)



[[193.22900390625, 249.19290161132812, 196.61345418294272], [144.9337158203125, 305.85205078125, 148.34825303819446]]
6.621621621621621
[[193.16021728515625, 249.23521423339844, 196.5774100974754]]
6.621621621621621


KeyboardInterrupt: 

In [3]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np
from operator import itemgetter
import time
from kuka import Kuka
from openshowvar import *
from realsense_camera import *
import pyrealsense2

print('go')
robot = openshowvar(ip = '192.168.1.2', port = 7000)
kuka = Kuka(robot)

kuka.read_cartessian()

kuka.set_base(17)
kuka.set_tool(10)

# Set speed (%)
kuka.set_speed(100)
print('ready')


trajectory_arr = []
trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
trajectory_arr.append(np.array([0, 0, -50, 0, 0, 180]))
trajectory_arr = np.array(trajectory_arr)
# Отправка траектории роботу
kuka.lin_continuous(trajectory_arr) 
kuka.open_grip()

cap = RealsenseCamera()
model = YOLO("Battery_1.pt")

bounding_box_annotator = sv.BoundingBoxAnnotator()
#label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER)
percentage_bar_annotator = sv.BoundingBoxAnnotator()
fx=604.602
fy=604.162
Cx=320
Cy=240

while True:
    ret, frame, depth = cap.get_frame_stream()
    #frame_re=frame[100:370,:]
    result = model(frame, agnostic_nms=True, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    #print(detections) 
    coord_center=[]
    for i in range (len(detections)):
        x=(detections.xyxy[i][0]+detections.xyxy[i][2])/2
        y=(detections.xyxy[i][1]+detections.xyxy[i][3])/2
        Y_mm=(depth[int(y), int(x)])*((Cy-y)/fy)
        coord_center.append([x,y,Y_mm])
    coord_center=np.array(coord_center)
    if coord_center.size != 0:
        x_sort=sorted(coord_center,key=itemgetter(0))
        #print(x_sort)
        currentClass = detections.class_id[0]
        if x_sort[0][0]<=580 and x_sort[0][0]>=565:
            second_min=time.time()
            #print(second_min)
        if x_sort[0][0]<=100 and x_sort[0][0]>=85:
            second_max=time.time()
            second=second_max-second_min-0.4
            print(second)
            
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([0, 106+x_sort[0][2], -120, 0, 0, 180]))
            trajectory_arr = np.array(trajectory_arr)
            # Отправка траектории роботу
            kuka.lin_continuous(trajectory_arr)
            time.sleep(second)
            kuka.close_grip()
            if currentClass==1:
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([0, 106+x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, -100, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
                kuka.open_grip()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([-330, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, 0, -50, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr)
            if currentClass==0:
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([0, 106+x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([-330, 170, -100, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
                kuka.open_grip()
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([-380, 170, 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, x_sort[0][2], 0, 0, 0, 180]))
                trajectory_arr.append(np.array([0, 0, -50, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr)    
              
    
    
    for i in range (len(detections)):
        cv2.circle(frame, (int(coord_center[i][0]),int(coord_center[i][1])), 3, (118, 103, 154), 3)
        cv2.putText(frame, 'y_mm: {}'.format(coord_center[i][2]), (210, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
    frame = bounding_box_annotator.annotate(
        scene=frame,
        detections=detections
    )

    #frame = label_annotator.annotate(
    #    scene=frame,
    #    detections=detections
    #)
    frame = percentage_bar_annotator.annotate(
        scene=frame,
        detections=detections
    )
    cv2.line(frame, (100,0), (100, 480), (0, 255, 0), 5)
    cv2.line(frame, (575,0), (575, 480), (255, 0, 0), 2)
    cv2.imshow("yolov8", frame)

    if (cv2.waitKey(30) == 27):
        break

go
ready


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


3.8099215984344483


KeyboardInterrupt: 